In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [88]:
stops_with_zip_codes = pd.read_csv('data/bus_stops_with_zip_codes_open_new.csv')
stops = pd.read_csv('data/stops.csv')
# census_data = pd.read_csv('data/census_data.csv')
neighborhood_zipcode=pd.read_csv("data/neighborhoods_to_zipcodes.csv")
bus_stops = pd.merge(stops_with_zip_codes, stops[['stop_id', 'route_id']], on='stop_id', how='inner')
print(bus_stops['zip_code'].dtype)
print(neighborhood_zipcode['ZipCodes'].dtype)
bus_stops['zip_code'] = bus_stops['zip_code'].astype(str)
bus_stops['stop_id'] = bus_stops['stop_id'].astype(int)
bus_stops.head()

int64
object


,stop_id,latitude,longitude,zip_code,route_id
0,2,42.332324,-71.081252,2119,['01']
1,5,42.336339,-71.077070,2118,['191' 'SL4' 'SL5']
2,7,42.333043,-71.081108,2118,['19' 'CT3']
3,8,42.331202,-71.070724,2118,['10']
4,13,42.329962,-71.057625,2127,['10' '16' '17' '171' '09' '18' 'CT3']


In [89]:
neighborhood_zipcode.head()

,Neighborhood,ZipCodes
0,Allston,"02135, 02163, 02134, 02215"
1,Back Bay,"02115, 02118, 02215, 02116, 02199, 02108"
2,Beacon Hill,"02116, 02114, 02108, 02111"
3,Brighton,"02135, 02134, 02467"
4,Charlestown,"02129, 02114"


In [90]:
bus_stops['zip_code'] = bus_stops['zip_code'].astype(str).apply(lambda x: '0' + x if len(x) == 4 else x)
bus_stops.head()

,stop_id,latitude,longitude,zip_code,route_id
0,2,42.332324,-71.081252,02119,['01']
1,5,42.336339,-71.077070,02118,['191' 'SL4' 'SL5']
2,7,42.333043,-71.081108,02118,['19' 'CT3']
3,8,42.331202,-71.070724,02118,['10']
4,13,42.329962,-71.057625,02127,['10' '16' '17' '171' '09' '18' 'CT3']


In [91]:
result_df = pd.DataFrame()
for idx, row in bus_stops.iterrows():
    # Split the 'route_id', 'stop_id', and 'zip_code' lists
    routes = row['route_id'].split(' ')
    stop_ids = row['stop_id']
    zip_codes = row['zip_code']

    # Create separate rows for each route with corresponding stop IDs and zip codes
    for i, route in enumerate(routes):
        result_df = result_df.append({'Routes': route, 'Stop_ID': stop_ids, 'zip_code': zip_codes}, ignore_index=True)

In [92]:
result_df.head()

,Routes,Stop_ID,zip_code
0,['01'],2.0,02119
1,['191',5.0,02118
2,'SL4',5.0,02118
3,'SL5'],5.0,02118
4,['19',7.0,02118


In [93]:
df=result_df.copy()
df.head()

,Routes,Stop_ID,zip_code
0,['01'],2.0,02119
1,['191',5.0,02118
2,'SL4',5.0,02118
3,'SL5'],5.0,02118
4,['19',7.0,02118


In [94]:
df['Routes'] = df['Routes'].str.replace("'", "").str.replace("[", "").str.replace("]", "")
df

,Routes,Stop_ID,zip_code
0,01,2.0,02119
1,191,5.0,02118
2,SL4,5.0,02118
3,SL5,5.0,02118
4,19,7.0,02118
...,...,...,...
2039,30,96459.0,02131
2040,24,96459.0,02131
2041,34E,98618.0,02081
2042,34,99832.0,02026


In [95]:
grouped_data=df.groupby('Routes').agg({'Stop_ID': list, 'zip_code': list}).reset_index()
grouped_data

,Routes,Stop_ID,zip_code
0,01,"[2.0, 59.0, 62.0, 64.0, 67.0, 72.0, 75.0, 79.0...","[02119, 02118, 02119, 02119, 02163, 02139, 021..."
1,04,"[113.0, 114.0, 117.0, 190.0, 214.0, 224.0, 236...","[02114, 02222, 02201, 02201, 02210, 02110, 021..."
2,07,"[33.0, 879.0, 886.0, 889.0, 890.0, 892.0, 6564...","[02127, 02127, 02127, 02210, 02210, 02205, 021..."
3,08,"[64.0, 111.0, 121.0, 123.0, 134.0, 142.0, 899....","[02119, 02125, 02125, 02125, 02125, 02125, 022..."
4,09,"[13.0, 21.0, 25.0, 33.0, 41.0, 45.0, 148.0, 15...","[02127, 02127, 02127, 02127, 02127, 02127, 021..."
...,...,...,...
147,CT3,"[7.0, 13.0, 128.0, 1784.0, 1799.0, 1805.0, 508...","[02118, 02127, 01125, 02120, 02120, 02215, 021..."
148,SL1,"[892.0, 17091.0, 17093.0, 17094.0, 17095.0, 17...","[02205, 02128, 02128, 02128, 02128, 02210, 021..."
149,SL2,"[30250.0, 31255.0, 74611.0, 74612.0, 74613.0, ...","[02127, 02210, 02210, 02210, 02210, 02210, 022..."
150,SL4,"[5.0, 55.0, 64.0, 5095.0, 5098.0, 6538.0, 6565...","[02118, 02118, 02119, 02118, 02118, 02111, 021..."


In [96]:
grouped_data.to_csv('data/routeTOzipcode_new.csv')

In [97]:
neighborhood_zipcode['ZipCodes'] = neighborhood_zipcode['ZipCodes'].str.split(',')
exploded_neighborhoods = neighborhood_zipcode.explode('ZipCodes')

In [98]:
# Explode the columns with multiple zip codes in both DataFrames
exploded_routes = grouped_data.explode('zip_code')

In [99]:
exploded_routes.head()

,Routes,Stop_ID,zip_code
0,01,"[2.0, 59.0, 62.0, 64.0, 67.0, 72.0, 75.0, 79.0...",02119
0,01,"[2.0, 59.0, 62.0, 64.0, 67.0, 72.0, 75.0, 79.0...",02118
0,01,"[2.0, 59.0, 62.0, 64.0, 67.0, 72.0, 75.0, 79.0...",02119
0,01,"[2.0, 59.0, 62.0, 64.0, 67.0, 72.0, 75.0, 79.0...",02119
0,01,"[2.0, 59.0, 62.0, 64.0, 67.0, 72.0, 75.0, 79.0...",02163


In [100]:
exploded_neighborhoods.head()

,Neighborhood,ZipCodes
0,Allston,02135
0,Allston,02163
0,Allston,02134
0,Allston,02215
1,Back Bay,02115


In [102]:
# Merge exploded DataFrames based on the exploded zip_code column
merged_data = pd.merge(exploded_routes, exploded_neighborhoods, left_on='zip_code', right_on='ZipCodes')

# Display the resulting merged data
print(merged_data)

    Routes                                            Stop_ID zip_code  \
0       01  [2.0, 59.0, 62.0, 64.0, 67.0, 72.0, 75.0, 79.0...    02115   
1       01  [2.0, 59.0, 62.0, 64.0, 67.0, 72.0, 75.0, 79.0...    02115   
2       01  [2.0, 59.0, 62.0, 64.0, 67.0, 72.0, 75.0, 79.0...    02115   
3       01  [2.0, 59.0, 62.0, 64.0, 67.0, 72.0, 75.0, 79.0...    02115   
4       01  [2.0, 59.0, 62.0, 64.0, 67.0, 72.0, 75.0, 79.0...    02115   
..     ...                                                ...      ...   
893     52  [774.0, 818.0, 820.0, 900.0, 903.0, 8394.0, 84...    02132   
894     52  [774.0, 818.0, 820.0, 900.0, 903.0, 8394.0, 84...    02132   
895     52  [774.0, 818.0, 820.0, 900.0, 903.0, 8394.0, 84...    02132   
896     52  [774.0, 818.0, 820.0, 900.0, 903.0, 8394.0, 84...    02132   
897     52  [774.0, 818.0, 820.0, 900.0, 903.0, 8394.0, 84...    02132   

      Neighborhood ZipCodes  
0         Back Bay    02115  
1           Fenway    02115  
2    Jamaica Plain   

In [103]:
merged_data.columns

Index(['Routes', 'Stop_ID', 'zip_code', 'Neighborhood', 'ZipCodes'], dtype='object')

In [128]:
merged_data.head()

,Routes,Stop_ID,zip_code,Neighborhood,ZipCodes
0,01,"[2.0, 59.0, 62.0, 64.0, 67.0, 72.0, 75.0, 79.0...",02115,Back Bay,02115
1,01,"[2.0, 59.0, 62.0, 64.0, 67.0, 72.0, 75.0, 79.0...",02115,Fenway,02115
2,01,"[2.0, 59.0, 62.0, 64.0, 67.0, 72.0, 75.0, 79.0...",02115,Jamaica Plain,02115
3,01,"[2.0, 59.0, 62.0, 64.0, 67.0, 72.0, 75.0, 79.0...",02115,Longwood,02115
4,01,"[2.0, 59.0, 62.0, 64.0, 67.0, 72.0, 75.0, 79.0...",02115,Roxbury,02115


In [104]:
result = merged_data.groupby('Neighborhood').agg({
    'Routes': lambda x: list(set(x))}).reset_index()

In [105]:
result = result.rename(columns={ 'Routes': 'Routes Serviced'})

In [106]:
result.shape

(23, 2)

In [107]:
result.head()

,Neighborhood,Routes Serviced
0,Allston,"[51, 66, 70, 65, 86, 193, 57, 501]"
1,Back Bay,"[CT2, 39, 21, 15, 23, 08, 55, 45, 09, 22, 29, ..."
2,Beacon Hill,"[09, 39, 43, 10]"
3,Brighton,"[51, 66, 70, 65, 86, 193, 57, 501]"
4,Charlestown,"[93, 92, 194]"


In [108]:
result.to_csv('data/NeighbourhoodTORoutesServiced_new.csv')

In [124]:
result['Routes Serviced']=result['Routes Serviced'].astype(str)

In [125]:
import ast
routes_to_neighborhoods = {}

for index, row in result.iterrows():
    neighborhood = row["Neighborhood"]
    routes_serviced = eval(row["Routes Serviced"])  # Evaluate the string as a list
    for route in routes_serviced:
        if route not in routes_to_neighborhoods:
            routes_to_neighborhoods[route] = [neighborhood]
        else:
            routes_to_neighborhoods[route].append(neighborhood)

# Create a DataFrame from the routes_to_neighborhoods dictionary
df_routes_to_neighborhoods = pd.DataFrame(routes_to_neighborhoods.items(), columns=["Route", "Neighborhoods"])

In [132]:
grouped_data['zip_code'] = grouped_data['zip_code'].astype(str)
merged_data['zip_code'] = merged_data['zip_code'].astype(str)

In [133]:
# Get unique zip codes from each DataFrame
unique_zipcodes_df1 = grouped_data['zip_code'].unique()
unique_zipcodes_df2 = merged_data['zip_code'].unique()

# Convert unique zip codes to sets for easier comparison
set_unique_zipcodes_df1 = set(unique_zipcodes_df1)
set_unique_zipcodes_df2 = set(unique_zipcodes_df2)

# Calculate the unique zip codes in each DataFrame
unique_in_df1 = set_unique_zipcodes_df1 - set_unique_zipcodes_df2
unique_in_df2 = set_unique_zipcodes_df2 - set_unique_zipcodes_df1

# Display the unique zip codes in each DataFrame and their count
print("Unique zip codes in DataFrame 1:", unique_in_df1)
print("Count of unique zip codes in DataFrame 1:", len(unique_in_df1))

print("\nUnique zip codes in DataFrame 2:", unique_in_df2)
print("Count of unique zip codes in DataFrame 2:", len(unique_in_df2))


Unique zip codes in DataFrame 1: {"['02215', '02215', '02215', '02120', '02445', '02447', '02467', '02447', '02445', '02120', '02215', '02215', '02467', '02120', '02459', '02467']", "['02124', '02124', '02124', '02121', '02119', '02121', '02121', '02124', '02124', '02215', '02119', '02120', '02120', '02215', '02121', '02120', '02120', '02215', '02119', '02120', '02120', '02115', '02121', '02121', '02115', '02115', '02115', '02119']", "['02118', '02127', '02127', '02127', '02127', '02127', '02127', '02116', '02116', '02216', '02119', '02119', '02121', '02121', '02118', '01125', '02116', '02121', '02125', '02119']", "['02126', '02130', '02137', '02137', '02131', '02131', '02131', '02131', '02136', '02136', '02136', '02136', '02136', '02136', '02136']", "['02119', '02125', '02125', '02125', '02125', '02125', '02215', '02215', '02215', '02115', '02120', '02120', '02215', '02118', '01125', '02115', '02115', '02125']", "['01730', '01865', '01803', '01805', '01805']", "['01960', '01960', '019